# Create Prediction with Segmentation Model

## Import Dependecies

In [12]:
from __future__ import absolute_import, division, print_function
import logging, os, sys

# Enable logging
logging.basicConfig(format='[%(levelname)s] %(message)s', level=logging.INFO, stream=sys.stdout)

# Re-import packages if they change
%load_ext autoreload
%autoreload 2

# Recursion Depth
import sys
sys.setrecursionlimit(10000)

# Intialize tqdm to always use the notebook progress bar
import tqdm
tqdm.tqdm = tqdm.tqdm_notebook

# Third-party libraries
import comet_ml
import numpy as np
import pandas as pd
import nilearn.plotting as nip
import matplotlib.pyplot as plt
import nibabel as nib
import numpy as np
import collections
import torch
import git


# Project utils

import aneurysm_utils
from aneurysm_utils import evaluation, training,preprocessing

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load Environment

In [3]:
if "workspace" in os.getcwd():
    ROOT = "/workspace"
elif "/group/cake" in os.getcwd(): 
    ROOT = "/group/cake"

In [ ]:
env = aneurysm_utils.Environment(project="our-git-project", root_folder=ROOT)
env.cached_data["comet_key"] = "EGrR4luSis87yhHbs2rEaqAWs" 

## Load Data

In [5]:
preprocessing_params = {
    'min_max_normalize': True,
    'mean_std_normalize': False,
    'smooth_img': False, # can contain a number: smoothing factor
    'intensity_segmentation': 0.2
}


In [8]:
#Creates list of mri_imgs to create predictions
#case_list_shuffled is the list of the cases of the test set after the train/test split with random seed 1 
#should be changed with other split or other test set

from aneurysm_utils import postprocessing
case_list_shuffled=["A130_R","A118","A120","A115","A133","A073","A072","A077","A064"]
mri_imgs_test=[]

for cases in case_list_shuffled:
    file_path=f"../../../../data/training/{cases}_orig.nii.gz"
    mri_imgs_test.append(nib.load(file_path).get_fdata())

    #preprocess the images 
mri_imgs_test=postprocessing.resample(mri_imgs_test,(70,70,60))
mri_imgs_test=preprocessing.preprocess(env,mri_imgs_test,preprocessing_params)

[INFO] Preprocessing: Min Max Normalize...
[INFO] Preprocessing: Intensity Segmentation...


In [9]:
#loads labels 
from aneurysm_utils import postprocessing
case_list_shuffled=["A130_R","A118","A120","A115","A133","A073","A072","A077","A064"]
labels_test=[]

for cases in case_list_shuffled:
    file_path=f"../../../../data/training/{cases}_masks.nii.gz"
    labels_test.append(nib.load(file_path).get_fdata())
#resampling the labels to the correct size
labels_test=postprocessing.resample(labels_test,(70,70,60))


## Predict with Trained Model

In [10]:
from aneurysm_utils.utils.pytorch_utils import predict
from aneurysm_utils.models.pointnet  import SegNet 

In [14]:
#loads the trained modelfile
model = SegNet(dropout=0.38, num_classes=2, sample_rate1 =0.2263, sample_rate2 =0.2941,start_radius =7.254)
PATH = "/group/cake/our-git-project/models/SegNet_SegNet_3000.pt"

device = torch.device('cpu')

#test = model()
model.load_state_dict(torch.load(PATH, map_location=device))

<All keys matched successfully>

In [15]:
#Creates the dataset of the images and labels
from aneurysm_utils.utils import pytorch_utils
from torch.utils.data.dataloader import DataLoader
test_dataset = pytorch_utils.PyTorchGeometricDataset(
                mri_images=mri_imgs_test,
                labels=labels_test,
                root="/group/cake/our-git-project/processed",
                split="test",
                force_processing=True            
            )

del mri_imgs_test,labels_test

Processing...
Done!
/group/cake/our-git-project/processed/processed/test.pt


In [ ]:
#creates the predictions
from torch_geometric.data import DataLoader as DataLoaderGeometric
test_loader = DataLoaderGeometric(
            test_dataset,
            batch_size=1,  # TODO: use fixed batch size of 5
            shuffle=False,
            num_workers=0,
            pin_memory=True,
        )
predictions = predict(model, test_loader, apply_softmax=False,ispointnet=True,dimension=(70, 70, 60))

In [ ]:
#saves predicted masks in prediction folder
np.save("/group/cake/our-git-project/predictions/pointnet_2.npy",predictions)